## Imports

In [1]:
import os, json
from datetime import datetime

from scripts.ollama_handler import OllamaMediaAnalysis
from scripts.file_analyzer import *

import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

## Parameters

In [2]:
# Set model for ollama
# ollama.list() # to get all models
MODEL = "granite3.1-moe:3b-instruct-q8_0" #"granite3.1-moe" #"granite3.1-dense:8b-instruct-q8_0" #"granite3.1-dense:8b"
#SYSTEM_PROMPT = f"You are a senior researcher, working on a media analysis of articles published in arabic newspapers about ChatGPT and the effect of Artificial Intelligence on society. For your answers only focus on topics that were mentioned in the text without adding any further information. Before answering, thoroughly think about the task, the content provided and build your answer with chain of thought reasoning."

SYSTEM_PROMPT = (
    "You are a senior researcher conducting a media analysis of Arabic newspaper articles "
    "about ChatGPT and the societal effects of Artificial Intelligence. Your role is to focus "
    "exclusively on the topics mentioned in the provided text, without introducing external information. "
    "Before responding, carefully analyze the task, thoroughly evaluate the content of the articles, "
    "and construct your answer using a clear chain of thought reasoning approach."
)

# MAY TAKE LONG TIME! Whether all files should be processed (indicates Ollama interactions)
PROCESS_DOCUMENTS = True

# Load spacy model
SPACY_MODEL = "en_core_web_lg"


# Set folder paths
DOCUMENTS_FOLDER = "docs"
FILES_FOLDER = os.path.join(DOCUMENTS_FOLDER, "DOCX") # PDFs
OUTPUT_FOLDER = os.path.join(DOCUMENTS_FOLDER, "Processed", MODEL)
EXPORT_FILE_NAME = f"{datetime.now().strftime("%y%m%d")}-{MODEL}-processed_documents.pkl"

## Helper functions

In [3]:
def flatten_dict(d: dict):
    """Recursively flatten a dictionary with nested lists."""
    flattened_dict = dict()
    for k, v in d.items():
        flat_list = []
        if isinstance(v, list):
            flat_list.extend(flatten_list(v))
        elif isinstance(v, dict):
            flat_list.extend(flatten_dict(v))
        
        flattened_dict[k] = flat_list
    return flattened_dict

def flatten_list(l: list):
    """Recursively flatten a nested list."""
    flat_list = []
    for item in l:
        if isinstance(item, list):
            flat_list.extend(flatten_list(item))
        else:
            flat_list.append(item)
    return flat_list

## Initialize File Analyzer
Handes all files, folder processing, text extraction, question answering

In [4]:
# Define questions for the analyzer
questions =  [
    "How do the media in this article frame the public discussion about ChatGPT? Are there certain **metaphors** that keep cropping up?",
    "Which role does or might the Arabic World play in the development of Artificial Intelligence? Answer with 'Not mentioned' if not applicable.",
    "Which use cases of Artificial Intelligence are helpful for the Arabic world based on this article?",
    "What is the final message of the article that the author wants to convey? Keep your answer short and precise!"]

In [5]:
# Initialize llm as instance of OllamaMediaAnalysis
llm = OllamaMediaAnalysis(model_name=MODEL, system_prompt=SYSTEM_PROMPT, debug=True)

# Initialize analyzer
analyzer = FileAnalyzer(ollama_handler=llm, entity_collection="spacy", spacy_model=SPACY_MODEL, file_name=EXPORT_FILE_NAME, output_folder=OUTPUT_FOLDER, questions=questions, debug=False, speed_debug=False)

In [6]:
if PROCESS_DOCUMENTS:
    # Process the documents in the folder where the PDFs are
    documents = analyzer.process_folder(FILES_FOLDER, file_types=(".docx"))

    # Save documents to the output folder
    analyzer.save_documents(documents)

01:04:44 Analyzing file from folder: ChatGPT AI grows more powerful as we become more predictable_standardizedlayout.docx
01:04:44	 Adding paragraph 1/11 with 32 characters
01:04:44	 Adding paragraph 2/11 with 19 characters
01:04:44	 Adding paragraph 3/11 with 13 characters
01:04:44	 Adding paragraph 4/11 with 19 characters
01:04:44	 Adding paragraph 5/11 with 36 characters
01:04:44	 Adding paragraph 6/11 with 20 characters
01:04:44	 Adding paragraph 7/11 with 11 characters
01:04:44	 Adding paragraph 8/11 with 0 characters
01:04:44	 Adding paragraph 9/11 with 0 characters
01:04:44	 Adding paragraph 10/11 with 0 characters
01:04:44	 Adding paragraph 11/11 with 5560 characters
01:04:44	 Create Docx document <ChatGPT AI grows mor...> with content of length 5539
01:04:44 Initialized Document: <ChatGPT AI grows more powerful as we become more predictable>
01:04:44	 Generating tokenized content
01:04:44	 Generating short summary
Validation Error: 1 validation error for ShortSummary
short_sum

In [7]:
# Load already analyzed documents
analyzer.load_documents(os.path.join(OUTPUT_FOLDER, EXPORT_FILE_NAME), load_latest=False)

01:27:30 Initialized Document: <ChatGPT AI grows more powerful as we become more predictable>
01:27:30 Initialized Document: <ChatGPT is the ‘Netscape moment' for artificial intelligence'>
01:27:30 Initialized Document: <ChatGPT outperforms copywriters in STEP Conference's outdoor adverts>
01:27:30 Initialized Document: <AI is not smarter than humans>
01:27:30 Initialized Document: <No need to demonize ChatGPT but AI regulation is a must>
01:27:30 Initialized Document: <Is the Arab world ready for the uncertain age of AI-powered web tools>
01:27:30 Initialized Document: <‘I am not here to take your job,' ChatGPT tells Frankly Speaking host>
01:27:30 Initialized Document: <Will ChatGPT and AI have an impact on Saudi workforce productivity>


## Get insights of all documents

In [8]:
# Print a summary of every loaded file (Optional: with markings)
for x, doc in enumerate(analyzer):
    
    #result = llm.analyze_sentiment(doc.content))
    #result = doc.sentimentsd

    #doc.content = analyzer.clean_input(doc.content)
    #doc.content_tokens = analyzer.get_tokens(doc.content)
    #doc.sentiment = result.get("sentiment_value")
    #doc.sentiment_reason = result.get("sentiment_reason")
    print(doc.get_info(), end="\n"+"- "*50+"\n"*2)
    
    #print(doc.summary)
    #print(ensure_numbered_list(doc.summary))

Title: ChatGPT AI grows more powerful as we become more predictable
Short Summary: ": "The article underscores ChatGPT's swift global adoption as an indicator of the emerging AI era, while acknowledging its lack of genuine intelligence and ethical challenges such as bias reinforcement, suppression of dissenting views, and limitations in balancing creativity with constraints."
}
Summary:
1. **Rapid Adoption Rate**: The advent of ChatGPT, an AI large language model owned by OpenAI, has seen unprecedented speedy adoption with a million unique users within five days of its release.
 
2. **AI in Emerging Markets**: Despite being in their infancy, platforms like ChatGPT are rapidly gaining popularity among companies and governments globally, particularly in emerging markets, signaling an eagerness for AI technology.

3. **Limited AI Intelligence**: Unlike the human mind's capacity to develop language with minimal data from its environment, large language models such as ChatGPT analyze vast a

In [9]:
# OPTIONAL: Stores an updates version of the documents
#analyzer.save_documents(analyzer.all_documents)

### Export the files as word-docx and markdown files

In [10]:
# Export docx files with wordclouds
analyzer.export_docx_files()

In [11]:
# Write a markdown file for every document
analyzer.export_markdown_files()

## Analysis of all files

### Apply latent dirichlet allocation algorithm
Algorighm selects all topics out of the articles. LLM then adds a title that summarizes the topics into categories. 

Thereby, all different topics can be extracted out of **all** documents.

In [12]:
# Get the content of all documents
all_content_tokens = [doc.content_tokens for doc in analyzer.all_documents]


# Create a document-term matrix
max_df = 0.8   # means "ignore terms that appear in more than 90% of documents".
min_df = 0.1  # means "ignore terms that appear in less than 20% documents".

vectorizer = CountVectorizer(
    max_df=max_df,
    min_df=min_df,
    stop_words="english",
    ngram_range=(1, 2) # Capture multi-word expressions
)
doc_term_matrix = vectorizer.fit_transform(all_content_tokens)

# Apply LDA
lda = LatentDirichletAllocation(n_components=5, learning_method="batch", random_state=42, n_jobs=-1, learning_decay=0.5)
lda.fit(doc_term_matrix)

LatentDirichletAllocation(learning_decay=0.5, n_components=5, n_jobs=-1,
                          random_state=42)

In [13]:
# Function to generate unique topics
def get_unique_topics(model, vectorizer, top_n=10):
    unique_topics = {}
    for idx, topic in enumerate(model.components_):
        # Get the top features for the topic
        top_features = tuple(vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-top_n - 1:-1])
        
        # Use the tuple as a key to ensure uniqueness
        if top_features not in unique_topics:
            unique_topics[top_features] = idx

    return unique_topics

# Generate a title for each unique topic
def generate_topic_titles(llm, unique_topics):
    titles = {}
    
    for features, idx in unique_topics.items():
        # Create a prompt with the top features
        prompt = (
            "You are tasked with creating a concise, descriptive title for a topic derived from text analysis. "
            "The title should reflect the general theme or idea represented by the following features. "
            "Avoid listing all feature names explicitly, and ensure the title is engaging and informative. "
            "Aim for a maximum of 5 words. Here are the features: "
            f"{', '.join(features)}"
        )
        
        # Use the LLM to generate a title
        response = llm.ollama.generate(model=llm.model, prompt=prompt)
        titles[idx] = response["response"].split("\n")[0]
    return titles


# Get unique topics and their titles
unique_topics = get_unique_topics(lda, vectorizer, top_n=15)
topic_titles = generate_topic_titles(llm, unique_topics)


analyzer.analysis["LDA"] = {topic_titles[idx].replace('"', ""): topics for idx, topics in zip(topic_titles, unique_topics)}

# Print unique topics and their generated titles
for idx, (title, topics) in enumerate(analyzer.analysis["LDA"].items()):

    print(f"Topic {idx} - {title}:")
    print(f"Features: {', '.join(topics)}")

Topic 0 - AI-Driven Medical Threat Analysis: A Comprehensive Field Report by Expert Team:
Features: generate, question, ai tool, step, report, need, threat, medical, raise, team, field, article, explain, fear, accord
Topic 1 - Google-Arab Chatbot: Revolutionizing Text Analysis & AI Tech for Billion+ Web Search:
Features: say, google, answer, chatbot, billion, arab, openai, search, web, text, tech, base, people, way, program
Topic 2 - AI Model: Language Proficiency & Predictability - Saudi-Based AI Platform:
Features: model, language, ability, ai tool, example, reason, large, time, predictable, word, development, platform, saudi, point, train
Topic 3 - Insights from Language Model Training: Employing Khoury's Approach for News Relevance and Impact:
Features: language, say, job, training, impact, response, model, training datum, khoury, language model, employee, news, provide, information, source
Topic 4 - Harness Microsoft's Powerful Thinking Tool: Revolutionize Your Lifestyle with 'Rea

### Apply Term frequency inverse term frequency
This model iterates over each document and returns those words, that do not appear often in other documents. The top n words are then used to create a topic for every article!

- A high TF-IDF score (FROM_LOW_TO_HIGH = False) indicates that a word is both important within a document and rare across all document.
- A low TF-IDF score suggests that a word is either common in the document but rare overall, or vice versa.

By analyzing TF-IDF scores for a set of words, you can identify:
Important keywords in a document
Rare or unique words that distinguish one document from another
Words with varying levels of importance across different documents


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from itertools import product

# Define the parameter grid
param_grid = {
    'max_df': [0.8, 0.9, 1.0],
    'min_df': [1, 2, 3],
    'ngram_range': [(1, 1), (1, 2), (1, 3)],
    'norm': ['l1', 'l2'],
}

# Initialize variables to store the best combination and vectorizer
best_params = None
best_vectorizer = None
best_vocab_size = 0  # Use vocabulary size as a proxy for quality

# Iterate over all combinations of parameters
for params in product(*param_grid.values()):
    # Map parameter combinations to their names
    params_dict = dict(zip(param_grid.keys(), params))
    
    # Create and fit the TfidfVectorizer with the current parameters
    vectorizer = TfidfVectorizer(
        stop_words='english',
        max_df=params_dict['max_df'],
        min_df=params_dict['min_df'],
        ngram_range=params_dict['ngram_range'],
        norm=params_dict['norm']
    )
    tfidf_matrix = vectorizer.fit_transform(all_content_tokens)
    
    # Evaluate based on vocabulary size
    vocab_size = len(vectorizer.get_feature_names_out())
    print(f"Params: {params_dict}, Vocabulary Size: {vocab_size}")
    
    # Track the best combination
    if vocab_size > best_vocab_size:
        best_vocab_size = vocab_size
        best_params = params_dict
        best_vectorizer = vectorizer

# Print the best parameters and corresponding vocabulary size
print("\nBest Parameters:", best_params)
print("Best Vocabulary Size:", best_vocab_size)

# Use the best vectorizer to transform the data
tfidf_matrix = best_vectorizer.transform(all_content_tokens)

Params: {'max_df': 0.8, 'min_df': 1, 'ngram_range': (1, 1), 'norm': 'l1'}, Vocabulary Size: 1572
Params: {'max_df': 0.8, 'min_df': 1, 'ngram_range': (1, 1), 'norm': 'l2'}, Vocabulary Size: 1572
Params: {'max_df': 0.8, 'min_df': 1, 'ngram_range': (1, 2), 'norm': 'l1'}, Vocabulary Size: 5803
Params: {'max_df': 0.8, 'min_df': 1, 'ngram_range': (1, 2), 'norm': 'l2'}, Vocabulary Size: 5803
Params: {'max_df': 0.8, 'min_df': 1, 'ngram_range': (1, 3), 'norm': 'l1'}, Vocabulary Size: 10428
Params: {'max_df': 0.8, 'min_df': 1, 'ngram_range': (1, 3), 'norm': 'l2'}, Vocabulary Size: 10428
Params: {'max_df': 0.8, 'min_df': 2, 'ngram_range': (1, 1), 'norm': 'l1'}, Vocabulary Size: 586
Params: {'max_df': 0.8, 'min_df': 2, 'ngram_range': (1, 1), 'norm': 'l2'}, Vocabulary Size: 586
Params: {'max_df': 0.8, 'min_df': 2, 'ngram_range': (1, 2), 'norm': 'l1'}, Vocabulary Size: 821
Params: {'max_df': 0.8, 'min_df': 2, 'ngram_range': (1, 2), 'norm': 'l2'}, Vocabulary Size: 821
Params: {'max_df': 0.8, 'min_df'

In [15]:
# Create a TfidfVectorizer object
min_df = int(len(analyzer.all_documents)//2)
tfidf_vectorizer = TfidfVectorizer(stop_words='english', norm="l2", max_df=0.8, min_df=min_df, ngram_range=(1,3)) 

# Fit and transform the documents into a TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(all_content_tokens)

# Get the feature names (i.e., words)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Convert the TF-IDF matrix to a DataFrame for better readability
df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

# Show the TF-IDF values for each term in each document
#print("TF-IDF Matrix:")
#print(df)

# Display the most important words (top N) for each document
TOP_N = 15
FROM_LOW_TO_HIGH = False

analyzer.analysis["TFIDF"] = dict()

for i, row in df.iterrows():
    doc = analyzer.all_documents[i]
    print(f"\nTop {TOP_N} terms for Document {i + 1}: <{doc.title}>")
          
    # Generate a title for each unique topic
    top_terms = row.sort_values(ascending=FROM_LOW_TO_HIGH).head(TOP_N*2)
    
    indices = top_terms.index
    values = top_terms.values
    
    # Create a prompt with the top features
    prompt = (
        "Generate a concise and meaningful title, exactly four words long, that summarizes the following features. "
        "The title should capture the main theme or topic of these features. "
        "Example outputs: 'Language Revolution', 'Shaping Future Technology Trends', 'Global Knowledge Network'."
        f"Features: {'\n'.join([f'{str(indices[i])} - {values[i]}' for i in range(len(indices))])}"
        )
            
    # Use the LLM to generate a title
    title = llm.ollama.generate(model=llm.model, prompt=prompt)["response"]
    
    analyzer.analysis["TFIDF"].update({doc: {"title": title, "terms": top_terms[:TOP_N]}})
    
    print(title, "\n", ", ".join(top_terms[:TOP_N].index))


Top 15 terms for Document 1: <ChatGPT AI grows more powerful as we become more predictable>
"AI-Driven Language Model Advancements: Ethical Applications and Societal Impact" 
 ai tool, language, development, openai, work, change, use, ability, chatgpt ai, produce, model, time, large, set, think

Top 15 terms for Document 2: <ChatGPT is the ‘Netscape moment' for artificial intelligence'>
"Saudi Arabia's Language Technology Challenge: A Comprehensive Approach to Language Development and Value Generation" 
 model, example, train, text, saudi, word, generate, language, value, challenge, require, research, base, point, learn

Top 15 terms for Document 3: <ChatGPT outperforms copywriters in STEP Conference's outdoor adverts>
"AI-Driven Job and Chatbot Enhancement: A New Era of Expertise" 
 job, chatbot, ai tool, use, need, explain, think, easy, continue, replace human, add, say, make, look, tell

Top 15 terms for Document 4: <AI is not smarter than humans>
"Microsoft's Chatbot Skill Revolut

In [16]:
content = json.dumps({doc.title: doc.content for doc in analyzer})

# Iterates over each question, provides answers to LLM and let them summarize
for question in questions:
    content = {doc.title: doc.answers.get(question) for doc in analyzer}
    response = llm.answer_question(text=json.dumps(content), question=question, multiple_articles=True)
    analyzer.analysis[question] = response

01:28:16	 Answering question <How do the media in this artic...>
01:28:31	 Answering question <Which role does or might the A...>
01:28:33	 Answering question <Which use cases of Artificial ...>
01:28:41	 Answering question <What is the final message of t...>


In [17]:
answers_questions = dict()
# Iterates over each question, provides answers to LLM and let them summarize
topic_question_all = (
    "Attached are the topics of every article. "
    "What **perspectives and aspects** are being widely covered? Which aspects are being ignored? "
    "In your answer consider topics such as, but not only, data privacy, costs/affordability, know-how, complexity, accuracy, accessibility, bias (towards age, gender, religion, sexuality), risks, opportunity, perception, limitations."
    "These are the topics of all arcticles: ")

# Example usage:
topic_clusters = {doc.title: [value for value in doc.topic_clusters.values()] for doc in analyzer}
content_topics = flatten_dict(topic_clusters)
response = llm.answer_question(text=json.dumps(content_topics), question=topic_question_all, multiple_articles=True)
analyzer.analysis["topic_question"] = response

01:28:46	 Answering question <Attached are the topics of eve...>


In [18]:
markdown = str()

lda = analyzer.analysis.get("LDA")
markdown += f"# Latent Dirichlet Allocation interpretation\n### Topics that are frequently mentioned in all articles\n- {'\n- '.join(lda.keys())}\n\n"


tfidf = [elem.get("title") for elem in flatten_list(analyzer.analysis.get("TFIDF").values())]
markdown += f"# TF-IDF interpretation for every article\n### Topics that appear in the article very often but not in others\n"

for id, doc in enumerate(analyzer):
    markdown += f"- <{doc.title}>: {tfidf[id]}\n"

markdown += "\n# Hypothesis/Questions\n"
for q_id, question in enumerate(questions + ["topic_question"]):
    answer = analyzer.analysis.get(question)
    markdown += f"## Question {q_id+1}\n"
    markdown += f"*{answer.get("question")}*\n\n{answer.get("answer")}\n\nReasoning: {answer.get("reasoning")}\n\n"
    
    
print(markdown)

markdown_path = os.path.join(analyzer.output_folder, "research summary.md")
analyzer.export_markdown(markdown_path, markdown)

# Latent Dirichlet Allocation interpretation
### Topics that are frequently mentioned in all articles
- AI-Driven Medical Threat Analysis: A Comprehensive Field Report by Expert Team
- Google-Arab Chatbot: Revolutionizing Text Analysis & AI Tech for Billion+ Web Search
- AI Model: Language Proficiency & Predictability - Saudi-Based AI Platform
- Insights from Language Model Training: Employing Khoury's Approach for News Relevance and Impact
- Harness Microsoft's Powerful Thinking Tool: Revolutionize Your Lifestyle with 'RealizeBoost'!

# TF-IDF interpretation for every article
### Topics that appear in the article very often but not in others
- <ChatGPT AI grows more powerful as we become more predictable>: "AI-Driven Language Model Advancements: Ethical Applications and Societal Impact"
- <ChatGPT is the ‘Netscape moment' for artificial intelligence'>: "Saudi Arabia's Language Technology Challenge: A Comprehensive Approach to Language Development and Value Generation"
- <ChatGPT outpe